### Ch9_Security_Databrick's Free Dolly with LangChain

#### System Requirement:
- The transformers library on a machine with GPUs

#### Install Required Dependencies:
- accelerate>=0.16.0
- transformers[torch]>=4.28.1
- torch>=1.13.1
- lanchain

In [1]:
!pip install "accelerate>=0.16.0,<1" "transformers[torch]>=4.28.1,<5" "torch>=1.13.1,<2" --quiet
#! install "accelerate>=0.16.0,<1" "transformers[torch]>=4.28.1,<5" "torch>=1.13.1,<2"
!pip install langchain>=0.0.139 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 80.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have 

#### Import Dependencies and Use pipeline to load a custom InstructionTextGenerationPipeline

The instruction following pipeline can be loaded using the `pipeline` function as shown below. This loads a custom `InstructionTextGenerationPipeline`.

`torch_dtype=torch.bfloat16` is generally recommended to be used if this type is supported in order to reduce memory usage.

Let's use `dolly-v2-12b` model.

In [2]:
import torch
from transformers import pipeline

generate_text = pipeline(model="databricks/dolly-v2-3b",
                         torch_dtype=torch.bfloat16,
                         trust_remote_code=True,
                         device_map="auto",
                         return_full_text=True)

config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

instruct_pipeline.py:   0%|          | 0.00/9.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/databricks/dolly-v2-3b:
- instruct_pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

You can then use the pipeline to answer instructions:

In [3]:
res = generate_text("What is difference between New York and New York state?")
print(res[0]["generated_text"])

What is difference between New York and New York state?
New York and New York state are both in United States. New York is a city, state is a state.


#### Import lanchain library functions to predict response based on provided Context

To use the pipeline with LangChain, you must set return_full_text=True, as LangChain expects the full text to be returned and the default for the pipeline is to only return the new text.

In [ ]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline

Template for an instrution with no input

In [ ]:
prompt = PromptTemplate(
    input_variables=["instruction"],
    template="{instruction}")

Template for an instruction with input

In [ ]:
prompt_with_context = PromptTemplate(
    input_variables=["instruction", "context"],
    template="{instruction}\n\nInput:\n{context}")

In [ ]:
hf_pipeline = HuggingFacePipeline(pipeline=generate_text)

llm_chain = LLMChain(llm=hf_pipeline, prompt=prompt)
llm_context_chain = LLMChain(llm=hf_pipeline, prompt=prompt_with_context)

In [ ]:
print(llm_chain.predict(instruction="Explain to me the difference between nuclear fission and fusion.").lstrip())

Nuclear fission creates a massive release of energy, in the form of atomic nuclei splitting. It is driven by a force called the weak force, which causes atomic nuclei to bind together. Nuclear fusion is a process in which two or more nuclei unite together to form one or more nuclei andrelease energy as heat. It occurs when two or more nuclei combine together to form a higher energy state, and the reaction is mediated by the strong force.


In [ ]:
context = """Lincoln's political career began in 1833 when he was elected to the Illinois House of Representatives.
Four years later, he was elected to the state Senate.  In 1847 he was elected Attorney General of Illinois, a position he held for ten years.
In 1859 he was elected to the United States Senate and served until 1861, when he was appointed to the US Supreme Court.
He served as Chief Justice from 1865-1868.
He returned to the US Senate in 1872 and served until 1875, when he was elected to his first Presidental office, serving as President from 1849-1850.
He became president once again in 1865-1869 and again in 1877-1881, this time of the University of Chicago.
He did not seek reelection in 1885.
During his time as president of the United States, he enacted the Civil War Amendments to the US Constitution, including the Thirteenth, Fourteenth and Fifteenth Amendments.
He also founded several historical landmarks including the University of Chicago and the World's Columbian Exposition in Chicago."""

print(llm_context_chain.predict(instruction="Summarize Lincoln's career?", context=context).lstrip())

Lincoln was a Civil War general, United States Senator, United States Supreme Court Justice and President of the United States.


In [ ]:
context="""There are times when the night sky glows with bands of color. The bands may
begin as cloud shapes and then spread into a great arc across the entire sky. They
may fall in folds like a curtain drawn across the heavens. The lights usually grow
brighter, then suddenly dim. During this time the sky glows with pale yellow, pink,
green, violet, blue, and red. These lights are called the Aurora Borealis. Some
people call them the Northern Lights. Scientists have been watching them for
hundreds of years. They are not quite sure what causes them. In ancient times
people were afraid of the Lights. They imagined that they saw fiery dragons in the
sky. Some even concluded that the heavens were on fire."""
print(llm_context_chain.predict(instruction="Summarize the night sky related to {context}", context=context).lstrip())

The night sky glows with bands of color that may begin as cloud shapes and then
spread into a great arc across the entire sky. The bands may fall in folds like a
curtain drawn across the heavens. The lights usually grow brighter, then suddenly
dim. These lights are called the Aurora Borealis.
